In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
import jax
import jax.numpy as jnp
import numpy as onp
import chex
from icecream import ic
from sentence_transformers import SentenceTransformer
from functools import partial
from typing import Optional, Tuple, Union, Any

from craftax.craftax_classic.constants import *
from gymnax.environments import environment, spaces
from sklearn.manifold import TSNE

from crafter_constants import blocks_labels, mobs_labels, inventory_labels

def embedding_crafter(embedding_model, next_obs):
    batch_size = next_obs.shape[0]
    maps, metadata = jnp.split(next_obs, [7 * 9 * 21], axis=1)
    
    # For each block/mob type, count how many are in frame
    maps = jnp.reshape(maps, [-1, 7, 9, 21])
    maps = jnp.transpose(maps, [0, 3, 1, 2])
    maps = jnp.reshape(maps, [-1, 21, 7 * 9])
    maps = maps.sum(axis=2)
    maps = jnp.round(maps).astype(jnp.int32)
    blocks, mobs = jnp.split(maps, [17], axis=1)

    # Extract and format metadata
    inventory, intrinsics, direction, light_level, is_sleeping = jnp.split(metadata, [12, 16, 20, 21], axis=1)
    inventory = jnp.round(inventory * 10.0).astype(jnp.int32)
    intrinsics *= 10.0

    health, food, drink, energy = jnp.split(intrinsics, [1, 2, 3], axis=1)

    # Construct embedding
    sentences = []
    for i in range(batch_size):
        blocks_gt0 = jnp.argwhere(blocks[i] > 0).reshape(-1)
        mobs_gt0 = jnp.argwhere(mobs[i] > 0).reshape(-1)
        inventory_gt0 = jnp.argwhere(inventory[i] > 0).reshape(-1)

        blocks_str = 'You see {}.'.format(', '.join([blocks_labels[b] for b in blocks_gt0]))
        mobs_str = 'You see {}.'.format(', '.join([mobs_labels[m] for m in mobs_gt0]))
        inventory_str = 'You have in your inventory {}.'.format(', '.join([inventory_labels[i] for i in inventory_gt0]))

        status = []
        if food[i].item() < 10.0 - 1e-4:
            status.append("hungry")
        if drink[i].item() < 10.0 - 1e-4:
            status.append("thirsty")
        if energy[i].item() < 10.0 - 1e-4:
            status.append("tired")
        status_str = 'You feel {}.'.format(', '.join(status))

        if health[i].item() < 5.0 - 1e-4:
            health_str = 'You are at low health.'
        elif health[i].item() < 10.0 - 1e-4:
            health_str = 'You are at moderate health.'
        else:
            health_str = 'You are at full health.'

        desc = ['You are playing a Minecraft-like survival game.', status_str, health_str]
        if len(blocks_gt0) > 0:
            desc.append(blocks_str)
        if len(mobs_gt0) > 0:
            desc.append(mobs_str)
        if len(inventory_gt0) > 0:
            desc.append(inventory_str)

        desc = ' '.join(desc)
        sentences.append(desc)

    embeddings = embedding_model.encode(sentences)
    embeddings = jnp.array(embeddings)
    return embeddings, sentences

/nas/ucb/arvindrajaraman/anaconda3/envs/diayn5/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from functools import partial
from jax import jit
from tqdm import tqdm

# @partial(jit, static_argnames=('batch_size',))
def obs_generator(key, batch_size=256):
    key, sb1, sb2, sb3, sb4, sb5, sb6 = jax.random.split(key, 7)
    direction = jax.random.randint(sb1, shape=(batch_size,), minval=0, maxval=4)
    direction = jax.nn.one_hot(direction, num_classes=4)
    block_mob_counts_1h = jax.random.bernoulli(sb2, 0.03, shape=(batch_size, 21 * 7 * 9))

    inventory = jnp.clip(jax.random.geometric(sb3, 0.5, shape=(batch_size, 12)) - 1, 0, 10)
    intrinsics = jax.random.randint(sb4, shape=(batch_size, 4), minval=0, maxval=11)
    light_level = jax.random.uniform(sb5, shape=(batch_size,), minval=0.0, maxval=10.0).reshape(batch_size, 1)
    is_sleeping = jax.random.randint(sb6, shape=(batch_size,), minval=0, maxval=2).reshape(batch_size, 1)
    
    obs = jnp.concatenate([block_mob_counts_1h, inventory, intrinsics, direction, light_level, is_sleeping], axis=1)
    return key, obs

In [4]:
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/nas/ucb/arvindrajaraman/anaconda3/envs/diayn5/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from models import Captioner
from structures import ReplayBuffer
import datetime
import optax

dummy_obs = jnp.zeros((1, 1345))
key = jax.random.PRNGKey(int(datetime.datetime.now().timestamp()))
key, init_key = jax.random.split(key)

captioner = Captioner(
    hidden_size=768
)
captioner_params = captioner.init(init_key, dummy_obs)
captioner_opt = optax.adam(learning_rate=0.0001)
captioner_opt_state = captioner_opt.init(captioner_params)

buffer = ReplayBuffer.create({
    'obs': onp.zeros((1345,)),
    'embed': onp.zeros((768,)),
}, size=50000)

In [6]:
@jit
def update_captioner(captioner_params, captioner_opt_state, obs, emdeds):
    def loss_fn(captioner_params, obs, embeds):
        preds = captioner.apply(captioner_params, obs)
        loss = optax.losses.squared_error(preds, emdeds).mean()
        return loss
    loss, grads = jax.value_and_grad(loss_fn)(captioner_params, obs, emdeds)
    captioner_updates, captioner_opt_state = captioner_opt.update(grads, captioner_opt_state)
    captioner_params = optax.apply_updates(captioner_params, captioner_updates)
    return captioner_params, captioner_opt_state, loss

In [16]:
TRAIN_STEPS = 10000
for i in tqdm(range(TRAIN_STEPS)):    
    sample = buffer.sample(1000)
    captioner_params, captioner_opt_state, loss = update_captioner(captioner_params, captioner_opt_state, sample['obs'], sample['embed'])
    if i % 100 == 0:
        print('Train step', i, 'Loss', loss)

  0%|          | 44/10000 [00:00<00:45, 217.62it/s]

Train step 0 Loss 2.01869e-05


  1%|▏         | 136/10000 [00:00<00:46, 211.45it/s]

Train step 100 Loss 2.0874786e-05


  2%|▏         | 219/10000 [00:01<01:03, 152.92it/s]

...

 98%|█████████▊| 9817/10000 [00:46<00:00, 197.89it/s]

Train step 9800 Loss 1.8634928e-05


 99%|█████████▉| 9930/10000 [00:47<00:00, 174.17it/s]

Train step 9900 Loss 1.8179519e-05


100%|██████████| 10000/10000 [00:47<00:00, 209.70it/s]


In [17]:
jnp.save('crafter_captioner.npy', captioner_params)

In [50]:
lparams = jnp.load('crafter_captioner.npy', allow_pickle=True)